In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [4]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3b
  atmpro = trp
  band = 3b
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.55, 1.5, 1.66)]
  wgt = [(0.8, 0.8, 0.8, 0.6, 0.6, 0.9)]
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312      1.0  66.806454                  NaN        NaN
0.000312      NaN        NaN                  1.0  29.224320
0.000750      2.0  30.207494                  2.0  25.037780
0.001052      3.0  28.256545                  3.0  26.637045
0.001476      4.0  25.342520                  4.0  27.110178
0.002070      5.0  22.175199                  5.0  26.137118
0.002904      6.0  19.227711                  6.0  23.685229
0.004074      7.0  16.723587                  7.0  20.093134
0.005714      8.0  14.726586                  8.0  16.182824
0.008015      9.0  13.201378                  9.0  12.953486
0.011243     10.0  12.057751                 10.0  11.085115
0.015771     11.0  11.192190                 11.0  10.512210
0.022122     12.0  10.521767                 12.0  10.601014
0.031031     13.0  10.093475                 13.0  10.839984
0.043528     14.0   9.450069                 14.0  10.509294
0.061057     15.0   8.351660                 15.0   9.292323
0.085645     16.0   7.344775                 16.0   7.800375
0.120136     17.0   6.667402                 17.0   6.563318
0.168516     18.0   6.253699                 18.0   5.855093
0.236378     19.0   6.090038                 19.0   5.770508
0.331549     20.0   6.202514                 20.0   6.167824
0.465100     21.0   6.816341                 21.0   7.030902
0.652400     22.0   6.516652                 22.0   6.809338
0.915100     23.0   5.077246                 23.0   5.225840
1.283650     24.0   3.914028                 24.0   3.712885
1.800600     25.0   3.128686                 25.0   2.789083
2.525700     26.0   2.506895                 26.0   2.387510
3.542800     27.0   1.955827                 27.0   2.153873
4.969550     28.0   1.572213                 28.0   1.806070
6.970850     29.0   1.402256                 29.0   1.477153
9.778100     30.0   1.262685                 30.0   1.257615
13.715850    31.0   1.074738                 31.0   1.203265
19.239350    32.0   0.842086                 32.0   1.061743
26.987250    33.0   0.698950                 33.0   0.834107
37.855300    34.0   0.648743                 34.0   0.655507
53.100050    35.0   0.551780                 35.0   0.570637
73.887500    36.0   0.434531                 36.0   0.522224
97.662500    37.0   0.339933                 37.0   0.405765
121.437500   38.0   0.247797                 38.0   0.249366
145.212500   39.0   0.170714                 39.0   0.125649
168.987500   40.0   0.126830                 40.0   0.067563
192.762500   41.0   0.096019                 41.0   0.043650
216.537500   42.0   0.071229                 42.0   0.034172
240.312500   43.0   0.047517                 43.0   0.026889
264.087500   44.0   0.012414                 44.0   0.005330
287.862500   45.0  -0.067561                 45.0  -0.071444
311.637500   46.0  -0.102247                 46.0  -0.103442
335.412500   47.0  -0.035700                 47.0  -0.027963
359.187500   48.0   0.020325                 48.0   0.028468
382.962500   49.0   0.037313                 49.0   0.040138
406.737500   50.0   0.045267                 50.0   0.044508
430.512500   51.0   0.049966                 51.0   0.047977
454.287500   52.0   0.052928                 52.0   0.050122
478.062500   53.0   0.054903                 53.0   0.051881
501.837500   54.0   0.056270                 54.0   0.052802
525.612500   55.0   0.057304                 55.0   0.053306
549.387500   56.0   0.057981                 56.0   0.053166
573.162500   57.0   0.058268                 57.0   0.052506
596.937500   58.0   0.057984                 58.0   0.051268
620.712500   59.0   0.056861                 59.0   0.048998
644.487500   60.0   0.054798                 60.0   0.046286
668.262500   61.0   0.051850                 61.0   0.042388
692.037500   62.0   0.048270                 62.0

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -14.281746   0.000000 -14.281746        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -14.020925   0.000188   
6.244000e-04 2     -14.283004   0.006200 -14.276805 -14.022118   0.003543   
8.759000e-04 3     -14.283608   0.007703 -14.275905 -14.022610   0.004781   
1.228600e-03 4     -14.284349   0.009625 -14.274724 -14.023234   0.006518   
1.723400e-03 5     -14.285253   0.012014 -14.273239 -14.024015   0.008888   
2.417400e-03 6     -14.286348   0.014933 -14.271416 -14.024985   0.012007   
3.390900e-03 7     -14.287668   0.018469 -14.269198 -14.026177   0.015931   
4.756500e-03 8     -14.289247   0.022754 -14.266493 -14.027628   0.020633   
6.672000e-03 9     -14.291125   0.027973 -14.263152 -14.029379   0.026055   
9.358900e-03 10    -14.293336   0.034387 -14.258950 -14.031465   0.032265   
1.312780e-02 11    -14.295913   0.042347 -14.253567 -14.033918   0.039668   
1.841450e-02 12    -14.298867   0.052309 -14.246557 -14.036750   0.049083   
2.583020e-02 13    -14.302156   0.064841 -14.237315 -14.039914   0.061561   
3.623230e-02 14    -14.305493   0.080615 -14.224877 -14.043119   0.078124   
5.082340e-02 15    -14.308571   0.100028 -14.208544 -14.046057   0.099228   
7.129060e-02 16    -14.311552   0.123257 -14.188295 -14.048852   0.124555   
1.000000e-01 17    -14.314544   0.151227 -14.163317 -14.051557   0.153791   
1.402710e-01 18    -14.317209   0.185698 -14.131511 -14.053783   0.187329   
1.967600e-01 19    -14.318834   0.229170 -14.089664 -14.054752   0.227481   
2.759970e-01 20    -14.317899   0.285397 -14.032502 -14.052908   0.279807   
3.871000e-01 21    -14.310627   0.359756 -13.950871 -14.044650   0.352731   
5.431000e-01 22    -14.283760   0.458850 -13.824910 -14.017347   0.455368   
7.617000e-01 23    -14.229987   0.573825 -13.656163 -13.964758   0.579124   
1.068500e+00 24    -14.161227   0.689585 -13.471642 -13.898061   0.702366   
1.498800e+00 25    -14.083461   0.811325 -13.272136 -13.822467   0.816045   
2.102400e+00 26    -13.995980   0.947548 -13.048433 -13.736337   0.929358   
2.949000e+00 27    -13.899854   1.102827 -12.797027 -13.640420   1.072899   
4.136600e+00 28    -13.805050   1.283168 -12.521882 -13.544045   1.279562   
5.802500e+00 29    -13.721577   1.509952 -12.211625 -13.457998   1.549957   
8.139200e+00 30    -13.641973   1.818492 -11.823482 -13.374043   1.874918   
1.141700e+01 31    -13.561301   2.228093 -11.333208 -13.287123   2.276354   
1.601470e+01 32    -13.494807   2.746933 -10.747873 -13.214450   2.859084   
2.246400e+01 33    -13.495966   3.391417 -10.104549 -13.215254   3.671107   
3.151050e+01 34    -13.592453   4.236914  -9.355538 -13.319442   4.669235   
4.420010e+01 35    -13.736991   5.356626  -8.380365 -13.477464   5.812700   
6.200000e+01 36    -13.907178   6.690255  -7.216923 -13.664187   7.202750   
8.577500e+01 37    -14.061919   8.068772  -5.993147 -13.830604   8.840069   
1.095500e+02 38    -14.134333   9.098546  -5.035787 -13.905678  10.058025   
1.333250e+02 39    -14.162555   9.824642  -4.337912 -13.930976  10.785690   
1.571000e+02 40    -14.195517  10.338390  -3.857127 -13.959506  11.168124   
1.808750e+02 41    -14.237001  10.737068  -3.499933 -13.994248  11.393165   
2.046500e+02 42    -14.289711  11.060198  -3.229514 -14.036265  11.558126   
2.284250e+02 43    -14.358229  11.329320  -3.028910 -14.087855  11.705966   
2.522000e+02 44    -14.452659  11.557572  -2.895087 -14.155688  11.849534   
2.759750e+02 45    -14.614721  11.754595  -2.860126 -14.283590  11.992448   
2.997500e+02 46    -15.004529  11.954130  -3.050399 -14.656943  12.164571   
3.235250e+02 47    -15.675121  12.336760  -3.338361 -15.343457  12.559728   
3.473000e+02 48    -16.454488  13.015586  -3.438902 -16.151491  13.289002   
3.710750e+02 49    -17

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')